In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import months_between, floor, current_date


spark = SparkSession.builder.getOrCreate()

In [0]:
catalog       = "workspace"
schema_bronze = "bronze"
schema_silver = "silver"
schema_gold   = "gold"

In [0]:
df_gold = (spark
           .table(f"{catalog}.{schema_silver}.pessoas")
           .withColumn("idade", floor(months_between(current_date(), F.col("data_nascimento")) / 12))
           .select(
               "codigo"
               , "nome_completo"
               , "idade"
               , "email"
               , "genero"
               , "pais"
               , "avatar"
               , "data_nascimento"
               )
           .withColumn("genero",
                                 F.when(
                                     F.col("genero") == "Male", "Masculino")
                                 .when(F.col("genero") == "Female", "Feminino")
                                 .otherwise("genero")
                                 )
           .withColumn("faixa_etaria",
                       F.when(F.col("idade") < 18, "18 -")
                        .when(F.col("idade") < 30, "18 a 29")
                        .when(F.col("idade") < 45, "30 a 44")
                        .when(F.col("idade") < 60, "45 a 59")
                        .otherwise("60 +")
                        )
           )

df_gold.display()

In [0]:
(df_gold
 .write
 .mode("overwrite")
 .format("delta")
 .saveAsTable(f"{catalog}.{schema_gold}.pessoas")
 )